In [1]:
%pip install pyspark
%pip install requests
%pip install gzip
%pip install io
%pip install pandas
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)
ERROR: No matching distribution found for gzip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
sources = [
    "https://datasets.imdbws.com/name.basics.tsv.gz",
    "https://datasets.imdbws.com/title.akas.tsv.gz",
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    "https://datasets.imdbws.com/title.crew.tsv.gz",
    "https://datasets.imdbws.com/title.episode.tsv.gz",
    "https://datasets.imdbws.com/title.principals.tsv.gz",
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
]

In [3]:
from pyspark.sql import SparkSession
import requests
import gzip
import io

# Initialize Spark session
spark = SparkSession.builder \
    .appName("IMDB Data Loading") \
    .getOrCreate()

# Load each dataset from sources
dataframes = {}
dataset_names = [
    "name_basics",
    "title_akas",
    "title_basics",
    "title_crew",
    "title_episode",
    "title_principals",
    "title_ratings"
]

for url, name in zip(sources, dataset_names):
    print(f"Loading {name} from {url}...")
    
    # Download and decompress the data
    response = requests.get(url)
    decompressed_data = gzip.decompress(response.content).decode('utf-8')
    
    # Convert to pandas DataFrame first 
    lines = decompressed_data.strip().split('\n')
    print(len(lines))
    dataframes[name] = lines
    

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 16:47:18 WARN Utils: Your hostname, MacBook-Pro-de-Ethan.local, resolves to a loopback address: 127.0.0.1; using 10.15.29.15 instead (on interface en0)
25/11/19 16:47:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 16:47:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Loading name_basics from https://datasets.imdbws.com/name.basics.tsv.gz...
14882055
Loading title_akas from https://datasets.imdbws.com/title.akas.tsv.gz...
53989582
Loading title_basics from https://datasets.imdbws.com/title.basics.tsv.gz...
12068991
Loading title_crew from https://datasets.imdbws.com/title.crew.tsv.gz...
12071057
Loading title_episode from https://datasets.imdbws.com/title.episode.tsv.gz...
9301107
Loading title_principals from https://datasets.imdbws.com/title.principals.tsv.gz...
95997473
Loading title_ratings from https://datasets.imdbws.com/title.ratings.tsv.gz...
1625494


In [ ]:
spark_dataframes = {}

for name, lines in dataframes.items():
    # Create RDD from the text data
    rdd = spark.sparkContext.parallelize(lines, numSlices=10)
    
    # Parse header and data
    header = lines[0].split('\t')
    data_lines = lines[1:]
    
    # Create DataFrame
    df = spark.read.option("header", "true") \
        .option("delimiter", "\t") \
        .option("nullValue", "\\N") \
        .csv(rdd)
    
    spark_dataframes[name] = df
    print(f"Loaded {name}: {df.count()} rows, {len(df.columns)} columns")

# Access individual dataframes like: spark_dataframes['name_basics'], spark_dataframes['title_ratings'], etc.
    

25/11/19 16:51:04 ERROR Inbox: An error happened while processing message in the inbox for LocalSchedulerBackendEndpoint
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:130)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1886)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1795)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1192)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:350)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:47)
	at org.apache.spark.serializer.JavaSerializerInst